Оцениваение: минимально в 2 раза больше обычного дз (если сделаете совсем много, возможны дополнительные бонусы).
На чуть больше половины баллов за проект достаточно правильно сделать CharCNN-BLSTM-CRF для NER и вариант модели Miwa & Bansal (2016) для RE.
Дедлайн 1 мая 9 утра (4 недели)

In [1]:
import brat_format
import os
from nltk.tokenize import sent_tokenize
import pickle 

import pandas as pd

In [2]:
ner_types = {"OUT", "ACT", "BIN", "CMP", "ECO", "INST", "MET", "SOC", "QUA"}
# rel_types = {"NNG", "NNT", "NPS", "FNG", "FNT", "FPS", "PNG", "PNT", "PPS", "GOL", "TSK"}

## Читаю данные

In [3]:
def read_data(ann_file):
    br_doc = brat_format.read_file(ann_file)
    segs = sent_tokenize(br_doc.txt_data, language="russian")

    count_chars = 0
    cnt_global_ners = 0
    ners_text = []
    for i in range(len(segs)):
        seg = segs[i]

        while br_doc.txt_data[count_chars] != seg[0]:
            count_chars += 1

        end_seg = count_chars + len(seg)
        ners_this_seg = []
        while br_doc.ners and br_doc.ners[0][1] <= end_seg:
            ent = br_doc.ners.pop(0)
            #print(ent)
            ners_this_seg.append((ent[0] - count_chars, ent[1] - count_chars, ent[2]))

        count_chars += len(seg)
        ners_text.append(ners_this_seg)

    #     print('-' * 20, 'NER', '-' * 20)
    #     for ner in ners_this_seg:
    #         print(ner[0], ner[1], seg[ner[0]: ner[1]], ner[2])

    #     print('\n\n\n')
    
    return segs, ners_text

## READ TRAIN_VAL

In [4]:
train_val_texts, train_val_ners = [], [] 

for i in range(1, 4):
    PATH = f'train_data/train_part_{i}/train_part_{i}/'
    texts = []
    ners = []

    list_files = os.listdir(PATH)
    ann_files = [x for x in list_files if x.endswith('ann')]
    
    for ann_file in ann_files:
        segs, ners_file = read_data(PATH + ann_file)
        
        texts += segs
        ners += ners_file
    
    train_val_texts += texts
    train_val_ners += ners
    
print(len(train_val_texts), len(train_val_ners))

10631 10631


In [5]:
i = 343

print(len(train_val_texts[i]), '\n\n', train_val_texts[i], '\n')

for ner in train_val_ners[i]:
    print(train_val_texts[i][ner[0]: ner[1]], ner[2])

295 

 Всего за 2015 год в районе:
 - введено   в эксплуатацию  - 18544 кв.м. площади, в т.ч. 
- 11090 кв. м. жилой, 
- выдано 153 разрешения на строительство, реконструкцию, капитальный ремонт объектов социальной сферы, жилых домов, газовых и электрических сетей,
- введено в эксплуатацию 50 объектов. 

введено   в эксплуатацию BIN
выдано BIN
разрешения на строительство, реконструкцию, капитальный ремонт объектов социальной сферы, жилых домов, газовых и электрических сетей ACT
введено в эксплуатацию BIN


## READ TEST

In [6]:
PATH = 'test_data/test_ner_only/'

list_files = os.listdir(PATH)
ann_files = [x for x in list_files if x.endswith('ann')]

test_texts = []
test_ners = []
for ann_file in ann_files:
    segs, ners_text = read_data(PATH + ann_file)

    test_texts += segs
    test_ners += ners_text
    
print(len(test_texts), len(test_ners))

19473 19473


In [7]:
train_val_ners_exist = set()

for spans in train_val_ners:
    for span in spans:
        train_val_ners_exist.add(span[2])
        
print(train_val_ners_exist)
print(ner_types - train_val_ners_exist)

{'CMP', 'ACT', 'INST', 'ECO', 'MET', 'SOC', 'QUA', 'BIN'}
{'OUT'}


In [8]:
test_ners_exist = set()

for spans in test_ners:
    for span in spans:
        test_ners_exist.add(span[2])
        
print(test_ners_exist)
print(ner_types - test_ners_exist)

{'CMP', 'ACT', 'INST', 'ECO', 'MET', 'SOC', 'QUA', 'BIN'}
{'OUT'}


## BIO

BIO разметка: B - begin, I - inner, O - outer. Преобразуем задачу разметки спанов в задачу классификации каждого слова.

0 = O = outer, 1 = B = begin, 2 = I = inner

In [9]:
tag_2_tag_id = {v: k + 1 for k, v in enumerate(ner_types - {'OUT'})}
tag_2_tag_id['OUT'] = 0

tag_id_2_tag = {v: k for k, v in tag_2_tag_id.items()}

tag_2_tag_id

{'CMP': 1,
 'ACT': 2,
 'INST': 3,
 'ECO': 4,
 'MET': 5,
 'SOC': 6,
 'QUA': 7,
 'BIN': 8,
 'OUT': 0}

In [10]:
from enum import IntEnum
from collections import namedtuple

from razdel import tokenize

from ipymarkup import show_box_markup
from ipymarkup.palette import palette

class Label(IntEnum):
    OUTER = 0
    BEGIN = 1
    INNER = 2

Sample = namedtuple("Sample", "text,tokens,spans,labels")

def text_span_to_sample(text, spans):
    labels = []
    tokens = list(tokenize(text))
    
    spans_copy = spans.copy()
    for token in tokens:
        label = tag_2_tag_id['OUT']
        
        if spans_copy:
            span_begin, span_end, tag = spans_copy[0]

            if token.start == span_begin or (token.start > span_begin and token.stop <= span_end):
                label = tag_2_tag_id[tag]

            if token.stop == span_end:
                spans_copy.pop(0)
            
        labels.append(label)
    
    return Sample(text, tokens, spans, labels)

In [11]:
# Train
train_val = []
for text, spans in zip(train_val_texts, train_val_ners):
    train_val.append(text_span_to_sample(text, spans))

In [12]:
# Test
test = []
for text, spans in zip(test_texts, test_ners):
    test.append(text_span_to_sample(text, spans))

In [13]:
i = 9346

show_box_markup(train_val[i].text, train_val[i].spans)
print(train_val[i].labels)
print(len(train_val))

[0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10631


In [14]:
i = 7

show_box_markup(test[i].text, test[i].spans)
print(test[i].labels)
print(len(test))

[0, 0, 0, 0, 4, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 5, 0, 5, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19473


In [15]:
len(train_val)

10631

In [16]:
import random
random.shuffle(train_val)

train = train_val[:9000]
val = train_val[9000:]

In [17]:
char_set = ["<pad>", "<unk>"] + list({ch for sample in train for token in sample.tokens for ch in token.text})
print(char_set)

['<pad>', '<unk>', 's', 'Ә', 'N', '<', '\uf0b7', 'H', '=', 'g', 'з', 'ф', 'ё', 'д', 'н', '8', 'С', 'R', 'ъ', '(', '7', 'п', 'х', 'O', '┤', 'A', 'П', 'т', 'v', 'u', '№', 'Х', 'б', 'Я', 'd', 'А', 'E', 'Щ', ',', 'Ю', '6', 'и', '1', '└', 'р', 'й', 'ж', 'э', 'x', 'l', 'Z', 'e', 'І', 'Ы', 'ч', 'Ц', '”', '┘', 'л', 'Ь', 'h', 'T', 'ь', '–', 'Ъ', 'с', '"', 'G', 'щ', '.', 'Э', 'w', 'Р', 'z', '[', '…', '@', '┌', '3', 'p', 'S', '»', '×', '*', 'V', 'В', 'o', 'F', 'к', 'Г', 'I', 'ш', 'Н', 'W', 'Ж', '┼', 'Т', '«', 'Д', 'y', '┐', 'Б', '%', 'ә', 'j', 'М', 'о', 'm', 'г', 'D', 'а', 'k', '─', '!', 'О', '├', ']', '2', 'е', 'f', '“', 'b', 'ы', 'X', '0', 'Ё', 'Σ', 'U', 'Е', '-', 'З', '9', '_', '/', ')', 'я', 'К', 'a', 'У', '∑', 'Ö', 'M', 'Й', 'i', 'ю', ';', ':', 'C', '┬', 'Л', '>', '+', 'м', 'B', 'K', '■', '•', 'ц', '5', 'Ч', 'r', 'у', '£', '—', '│', 'җ', 'P', 'c', '4', 'И', 't', 'Ф', 'Ш', 'в', 'L', 'n']


Для каждого слова сохраняем его символьный состав, а в остальном старый добрый пайплайн

In [18]:
stat = []

for i in range(len(train_val)):
    stat.append(len(train_val[i].text))

pd.Series(stat).describe(percentiles=[x/10 for x in range(10)])

count    10631.000000
mean       231.197065
std        286.775179
min          1.000000
0%           1.000000
10%         10.000000
20%         73.000000
30%        105.000000
40%        134.000000
50%        166.000000
60%        202.000000
70%        248.000000
80%        320.000000
90%        461.000000
max       7562.000000
dtype: float64

In [19]:
# import torch
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader, RandomSampler

# class NerCharsDataset(Dataset):
#     def __init__(self, samples, char_set, max_seq_len=500, max_char_seq_len=50):
#         assert len(samples) != 0
#         self.samples = []
#         self.tokens = []
#         self.texts = []
#         for sample in samples:
#             inputs = torch.zeros((max_seq_len, max_char_seq_len), dtype=torch.long)
#             for token_num, token in enumerate(sample.tokens[:max_seq_len]):
#                 for ch_num, ch in enumerate(token.text[:max_char_seq_len]):
#                     char_index = char_set.index(ch) if ch in char_set else char_set.index("<unk>")
#                     inputs[token_num][ch_num] = char_index
#             labels = torch.zeros((max_seq_len,), dtype=torch.long)
#             input_labels = [int(i) for i in sample.labels[:max_seq_len]]
#             labels[:len(input_labels)] = torch.LongTensor(input_labels)
#             self.samples.append((torch.LongTensor(inputs), torch.LongTensor(labels)))
#             self.tokens.append(sample.tokens[:max_seq_len])
#             self.texts.append(sample.text)
    
#     def __len__(self):
#         return len(self.samples)
    
#     def __getitem__(self, index):
#         return self.samples[index]

#     def get_tokens(self, index):
#         return self.tokens[index]
    
#     def get_text(self, index):
#         return self.texts[index]


# BATCH_SIZE = 128

# train_data = NerCharsDataset(train, char_set)
# train_sampler = RandomSampler(train_data)
# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

# val_data = NerCharsDataset(val, char_set)
# val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

# test_data = NerCharsDataset(test, char_set)
# test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [20]:
# for sample in train_loader:
#     inputs, labels = sample
#     print(inputs.size())
#     print(labels.size())
#     break

# # inputs: batch_size x num_words x num_chars
# # labels: batch_size x num_words

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import numpy as np

def get_next_gen_batch(samples, max_seq_len=500, max_char_seq_len=50, batch_size=128, shuffle=True):
    indices = np.arange(len(samples))
    if shuffle:
        np.random.shuffle(indices)
        
    batch_begin = 0
    while batch_begin < len(samples):
        batch_indices = indices[batch_begin: batch_begin + batch_size]
        batch = []
        batch_labels = []
        batch_max_len = 0
        for data_ind in batch_indices:
            sample = samples[data_ind]
            inputs = []
            for token in sample.tokens[:max_seq_len]:
                chars = [char_set.index(ch) if ch in char_set else char_set.index("<unk>") for ch in token.text][:max_char_seq_len]
                chars += [0] * (max_char_seq_len - len(chars))
                inputs.append(chars)
            batch_max_len = max(batch_max_len, len(inputs))
            inputs += [[0]*max_char_seq_len] * (max_seq_len - len(inputs))
            batch.append(inputs)
            labels = sample.labels[:max_seq_len]
            labels += [0] * (max_seq_len - len(labels))
            batch_labels.append(labels)
        batch_begin += batch_size
        batch = torch.cuda.LongTensor(batch)[:, :batch_max_len]
        labels = torch.cuda.LongTensor(batch_labels)[:, :batch_max_len]
        yield batch_indices, batch, labels

In [21]:
def train_gen_model(model, train_samples, val_samples, epochs_count=50, 
                    loss_every_nsteps=1000, lr=0.01, save_path="model_30_04.pth", device_name="cuda",
                    early_stopping=True):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss().cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (_, batch, batch_labels) in enumerate(get_next_gen_batch(train)):
            logits = model(batch) # Прямой проход
            logits = logits.transpose(1, 2)
            loss = loss_function(logits, batch_labels) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (_, batch, batch_labels) in enumerate(get_next_gen_batch(val)):
            logits = model(batch) # Прямой проход
            logits = logits.transpose(1, 2)
            val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if early_stopping and prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

C:\Users\Artem\AppData\Roaming\Python\Python36\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
# import torch
# from torch import nn

# class SuperSimpleModel(nn.Module):
#     def __init__(self, char_set_size, char_embedding_dim=16, classes_count=9, char_max_seq_len=50):
#         super().__init__()
        
#         self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
#         self.out_layer = nn.Linear(char_max_seq_len * char_embedding_dim, classes_count)

#     def forward(self, inputs):
#         projections = self.embeddings_layer.forward(inputs)
#         projections = projections.reshape(projections.size(0), projections.size(1), -1)
#         output = self.out_layer.forward(projections)
#         return output


# model = SuperSimpleModel(len(char_set))
# train_gen_model(model, train, val, epochs_count=30, early_stopping=False, lr=0.02)

In [44]:
from torch import nn

class CharFFLstmModel(nn.Module):
    def __init__(self, char_set_size, char_embedding_dim=8, classes_count=9, word_embedding_dim=32, lstm_embedding_dim=32, char_max_seq_len=50):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(char_set_size, char_embedding_dim)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(char_embedding_dim * char_max_seq_len, word_embedding_dim)
        self.relu = nn.ReLU()
        self.lstm_layer = nn.LSTM(word_embedding_dim, lstm_embedding_dim // 2, batch_first=True, bidirectional=True)
        self.out_layer = nn.Linear(lstm_embedding_dim, classes_count)

    def forward(self, inputs):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.reshape(projections.size(0), projections.size(1), -1)
        projections = self.relu(self.linear(projections))
        projections = self.dropout(projections)
        output, _= self.lstm_layer(projections)
        output = self.dropout(output)
        output = self.out_layer.forward(output)
        return output

In [45]:
model = CharFFLstmModel(len(char_set), classes_count=len(ner_types))
train_gen_model(model, train, val, epochs_count=150, early_stopping=False, lr=0.01, save_path="model_30_04_no_pl.pth")

Trainable params: 37865
Epoch = 0, Avg Train Loss = 0.0336, Avg val loss = 0.2767, Time = 10.78s
Epoch = 1, Avg Train Loss = 0.0244, Avg val loss = 0.1869, Time = 9.94s
Epoch = 2, Avg Train Loss = 0.0156, Avg val loss = 0.1799, Time = 9.99s
Epoch = 3, Avg Train Loss = 0.0141, Avg val loss = 0.1682, Time = 10.28s
Epoch = 4, Avg Train Loss = 0.0146, Avg val loss = 0.1740, Time = 9.91s
Epoch = 5, Avg Train Loss = 0.0137, Avg val loss = 0.1727, Time = 10.31s
Epoch = 6, Avg Train Loss = 0.0136, Avg val loss = 0.1501, Time = 10.08s
Epoch = 7, Avg Train Loss = 0.0134, Avg val loss = 0.1521, Time = 9.83s
Epoch = 8, Avg Train Loss = 0.0134, Avg val loss = 0.1459, Time = 10.27s
Epoch = 9, Avg Train Loss = 0.0135, Avg val loss = 0.1495, Time = 9.76s
Epoch = 10, Avg Train Loss = 0.0126, Avg val loss = 0.1496, Time = 10.29s
Epoch = 11, Avg Train Loss = 0.0125, Avg val loss = 0.1476, Time = 10.38s
Epoch = 12, Avg Train Loss = 0.0121, Avg val loss = 0.1260, Time = 10.34s
Epoch = 13, Avg Train Loss = 

In [30]:
! du -hs model_30_04_no_pl.pth

152K	model_30_04.pth


In [49]:
filepath = "model_30_04_no_pl.pth"

model = CharFFLstmModel(len(char_set))
model.load_state_dict(torch.load(filepath))

<All keys matched successfully>

In [55]:
model

CharFFLstmModel(
  (embeddings_layer): Embedding(176, 8)
  (dropout): Dropout(p=0.4, inplace=False)
  (linear): Linear(in_features=400, out_features=64, bias=True)
  (relu): ReLU()
  (lstm_layer): LSTM(64, 16, batch_first=True, bidirectional=True)
  (out_layer): Linear(in_features=32, out_features=9, bias=True)
)

# Оценка качества

In [52]:
indices, batch, batch_labels = next(get_next_gen_batch(test))

In [54]:
batch.device

device(type='cuda', index=0)

In [83]:
len(test)

19473

In [66]:
import evaluate_ners

MAX_SEQ_LEN = 500
test_sizz = len(test)

total_tp = 0
total_fp = 0
total_fn = 0

global_true = []
global_pred = []
labels_pred = []

device = torch.device("cuda")
model = model.to(device)

for batch_index, (indices, batch, batch_labels) in enumerate(get_next_gen_batch(test, shuffle=False)):
    BATCH_SIZE = indices.shape[0]
    logits = model(batch)
    
    predicted_labels = logits.argmax(dim=2).detach().cpu()
    labels_pred.append(predicted_labels)
    
    for i in range(BATCH_SIZE):
        num = batch_index * BATCH_SIZE + i
        if num == test_sizz:
            break
        
        #print('text: ', i)
        ners_model = []
        
        tokens = list(tokenize(test_texts[num]))
        tokens = tokens[:MAX_SEQ_LEN]

        indices = [i for i, x in enumerate(predicted_labels[i]) if x != 0]
        indices_tensor = torch.Tensor(indices).type(torch.int64)

        vals = predicted_labels[i].gather(dim=0, index=indices_tensor)
        tokens_selected = [tokens[i] for i in indices]

        pred_tag_id = None
        start = None
        stop = None
        for token, val in zip(tokens_selected, vals):
            tag_id = int(val.item())
            
            if tag_id == pred_tag_id and token.start == stop + 1:
                stop = token.stop
            else:
                if pred_tag_id is not None:
                    ners_model.append((start, stop, tag_id_2_tag[pred_tag_id]))
                    
                pred_tag_id = tag_id
                start = token.start
                stop = token.stop
                
        if (len(tokens_selected) != 0 and
            (len(ners_model) == 0 or ners_model[-1][1] != stop)):
            ners_model.append((start, stop, tag_id_2_tag[pred_tag_id]))
            
        global_true.append(test_ners[i])
        global_pred.append(ners_model)

        tp, fp, fn = evaluate_ners.cacl_ner_tp_fp_fn(test_ners[i], ners_model)
        total_tp += tp
        total_fp += fp
        total_fn += fn
        
    if batch_index % 100 == 0:
        print('batch_index: ', batch_index)
        
#     if batch_index == 1:
#         break
        
precision, recall = evaluate_ners.compute_precision_and_recall(total_tp, total_fp, total_fn)
f_measure = 2 * precision * recall / (precision + recall)

batch_index:  0
batch_index:  100


IndexError: list index out of range

In [75]:
batch.size()[0]

17

In [61]:
indices, batch, batch_labels = next(get_next_gen_batch(test))

In [62]:
device = torch.device("cuda")
model = model.to(device)

In [ ]:
BATCH_SIZE = indices.shape[0]
logits = model(batch)

predicted_labels = logits.argmax(dim=2).detach().cpu()
labels_pred.append(predicted_labels)

for i in range(BATCH_SIZE):
    num = batch_index * BATCH_SIZE + i
    if num == test_sizz:
        break

    #print('text: ', i)
    ners_model = []

    tokens = list(tokenize(test_texts[num]))
    tokens = tokens[:MAX_SEQ_LEN]

    indices = [i for i, x in enumerate(predicted_labels[i]) if x != 0]
    indices_tensor = torch.Tensor(indices).type(torch.int64)

    vals = predicted_labels[i].gather(dim=0, index=indices_tensor)
    tokens_selected = [tokens[i] for i in indices]

In [68]:
total_tp, total_fp, total_fn

(163, 118609, 81011)

In [69]:
precision, recall = evaluate_ners.compute_precision_and_recall(total_tp, total_fp, total_fn)
f_measure = 2 * precision * recall / (precision + recall)

print(precision, recall, f_measure)

0.002008032128514056 0.0013723773279897618 0.0016304402188590918


In [77]:
device = torch.device("cuda")
model = model.to(device)

indices, batch, batch_labels = next(get_next_gen_batch(test))
batch_size = batch.size()[0]
logits = model(batch)

predicted_labels = logits.argmax(dim=2).detach().cpu()
#labels_pred.append(predicted_labels)

In [82]:
ind = 9

for i, j in zip(train_val[ind].labels,  predicted_labels[ind]):
    print(i, int(j.item()))

8 0
0 0
0 0
0 0
0 0
3 3
3 3
3 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
8 0
0 0
0 0
5 0
5 0
5 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
4 0
0 0


In [25]:
classes_count=len(ner_types)

In [26]:
m = CRF(classes_count, batch_first=True)

In [27]:
emissions = torch.randn(BATCH_SIZE, 500, classes_count)

In [28]:
inputs, true_labels = next(iter(train_loader))

In [30]:
BATCH_SIZE

128

In [29]:
m(emissions, true_labels)

tensor(-173546.2031, grad_fn=<SumBackward0>)

In [31]:
173546 / 128

1355.828125

In [33]:
num_tags = 5  # number of tags is 5
model = CRF(num_tags)

In [34]:
seq_length = 3  # maximum sequence length in a batch
batch_size = 2  # number of samples in the batch
emissions = torch.randn(seq_length, batch_size, num_tags)
tags = torch.tensor([
            [0, 1], [2, 4], [3, 1]], dtype=torch.long)  # (seq_length, batch_size)
model(emissions, tags)

tensor(-12.0899, grad_fn=<SumBackward0>)

In [186]:
for i, (true, pred) in enumerate(zip(global_true, global_pred)):
    print(i, 'true: ', true, '\t', 'pred: ', pred)

0 true:  [(353, 363, 'ECO'), (395, 406, 'BIN')] 	 pred:  [(40, 74, 'INST'), (275, 288, 'INST'), (328, 338, 'BIN'), (341, 352, 'BIN'), (353, 363, 'ECO')]
1 true:  [(0, 9, 'BIN')] 	 pred:  [(0, 9, 'BIN'), (10, 19, 'ECO')]
2 true:  [] 	 pred:  []
3 true:  [(82, 101, 'INST'), (103, 134, 'INST'), (136, 188, 'INST')] 	 pred:  [(0, 8, 'ACT'), (12, 23, 'ACT'), (35, 48, 'ACT'), (113, 134, 'ECO'), (136, 145, 'ECO'), (146, 188, 'INST')]
4 true:  [] 	 pred:  []
5 true:  [] 	 pred:  []
6 true:  [] 	 pred:  [(62, 75, 'INST')]
7 true:  [(45, 52, 'ECO'), (66, 73, 'ECO'), (76, 83, 'ECO'), (192, 204, 'ECO'), (205, 212, 'MET'), (215, 232, 'MET'), (233, 247, 'ECO')] 	 pred:  [(45, 120, 'ECO'), (205, 239, 'ECO'), (240, 247, 'MET')]
8 true:  [(0, 10, 'CMP'), (11, 18, 'MET'), (19, 29, 'BIN'), (38, 49, 'ECO'), (68, 101, 'ECO')] 	 pred:  [(0, 29, 'ECO'), (38, 49, 'ECO'), (68, 101, 'ECO')]
9 true:  [(12, 30, 'MET'), (61, 67, 'CMP')] 	 pred:  [(12, 30, 'MET')]
10 true:  [(0, 51, 'MET'), (53, 64, 'MET'), (65, 98,